In [3]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
sns.set()
import gc
gc.collect()
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
from IPython.display import clear_output
#set_matplotlib_formats('retina')
#from tqdm import tqdm
#tqdm.pandas()
import numpy as np
from pyhive import presto
from datetime import datetime, timedelta
#from bson import ObjectId
from functools import reduce
from sklearn.cluster import KMeans
#from pymongo import MongoClient
import glob
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import json
#import re
#import dtale
#from h3 import h3
# import pandasql as ps

In [4]:
metabase_connection = presto.connect(
        #host='bi-trino-2.serving.data.production.internal',
        #host='bi-trino.serving.data.production.internal',
        #host='bi-presto.serving.data.production.internal',
        #host='prime-trino.serving.data.production.internal',
    
        host= 'presto-gateway.processing.data.production.internal', #'presto.processing.yoda.run', #'presto-gateway.processing.data.production.internal',#'presto-gateway.processing.data.production.internal',    #"presto.processing.yoda.run",
        port=80,
        protocol='http',
        catalog ='hive',
        username='praveen.u@rapido.bike')

In [ ]:
# select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
        
#         concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
#         concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch
#         from raw.kafka_dispatch_propagation_immutable
#         where yyyymmdd = cast({dt} as varchar)
#         -- where yyyymmdd = '20230314'

#             and data_orderid in (select distinct order_id from order_filtering)
#             and data_propagationtype in ('multicast')
#             and eventtype in ('rider_acknowledged')

        
#         union all
        
#         select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
        
#         concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
#         concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch
#         from(
#         select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,epoch,
#         row_number() over(partition by data_propagationBatchId,data_riderId,data_orderid order by epoch desc) as row,data

#         from raw.kafka_dispatch_propagation_immutable
#         where yyyymmdd = cast({dt} as varchar)
#         -- where yyyymmdd = '20230314'

#             and data_orderid in (select distinct order_id from order_filtering)
#             and data_propagationtype in ('multicast')
#             and eventtype in ('rider_busy','order_already_accepted','rider_accept_failed','rider_rejected','rider_accepted','rider_ineligible','rider_unreachable')
#             ) base
#         where row =1

In [ ]:
#orders which have lm <1km 



In [42]:
pings_dt1=pd.DataFrame()
for i in pd.date_range('20230619','20230619'):

    query="""with order_filtering as 
    (
        select *,
        case when rounded_last_mile<=1 then 'lm_less_than_1km'
        else 'lm_greater_than_1km' end as lm_flag
        from
        (
        select distinct yyyymmdd,order_id ,city_name, pickup_location_hex_8,drop_location_hex_8,
            case when distance_final_distance<0.5 then round(distance_final_distance)+1 
            else round(distance_final_distance) end as rounded_last_mile,
                CASE 
                    WHEN substr(quarter_hour,1,2) IN ('08','09','10','11') THEN 'morning_peak'
                    WHEN substr(quarter_hour,1,2) IN ('12','13','14','15','16') THEN 'afternoon'
                    WHEN substr(quarter_hour,1,2) IN ('17','18','19','20','21') THEN 'evening_peak'
                    WHEN substr(quarter_hour,1,2) IN ('00','01','02','03','04','05','06','07') THEN 'rest_morning'        
                    ELSE 'rest_evening' END AS time_period, order_status,quarter_hour

        from 
            orders.order_logs_snapshot
            where yyyymmdd = '{std}'
            and service_obj_service_name = 'Auto'
            and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)
            and lower(trim(city_name)) in ('hyderabad') 
            )
    ),
    rider_pings as (
        select yyyymmdd,data_orderid,data_riderId,
            count(distinct data_orderid) as Gross_orders,
            count(distinct case when event_type = 'rider_busy' then unique_ping_level_id end) as riderBusy_pings,
            count(distinct case when event_type = 'rider_rejected' then unique_ping_level_id end) as riderreject_Pings,
            count(distinct case when event_type = 'rider_accepted' then unique_ping_level_id end) as accepted_pings,
            count(distinct case when event_type = 'order_already_accepted' then unique_ping_level_id end) as already_accepted_pings,
            count(distinct case when event_type = 'rider_accept_failed' then unique_ping_level_id end) as rider_accept_failure_pings,        
            count(distinct case when event_type = 'rider_acknowledged' then unique_ping_level_id end) as rider_ack_pings
        from
        (
               select yyyymmdd,data_propagationBatchId,data_riderId,data_orderid,
                event_type_agg,eventtype as event_type,
                unique_ping_level_id,unique_id_order_level_id

                from
                (
            select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
            array_agg(eventtype) over (partition by data_orderid) as event_type_agg,
            concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
            concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch
            from raw.kafka_dispatch_propagation_immutable
            where yyyymmdd = '{std}'

                and data_orderid in (select distinct order_id from order_filtering)
                and data_propagationtype in ('multicast')
                and eventtype in ('rider_acknowledged')


            union all

            select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
            array_agg(eventtype) over (partition by data_orderid) as event_type_agg,
            concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
            concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch

            from(
            select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,epoch,
            row_number() over(partition by data_propagationBatchId,data_riderId,data_orderid order by epoch desc) as row,data

            from raw.kafka_dispatch_propagation_immutable
             where yyyymmdd = '{std}'

                and data_orderid in (select distinct order_id from order_filtering)
                and data_propagationtype in ('multicast')
                and eventtype in ('rider_busy','order_already_accepted','rider_accept_failed','rider_rejected','rider_accepted','rider_ineligible','rider_unreachable')
                ) base
            where row =1
                    )
                )
            group by 1,2,3

        )

    select  of.yyyymmdd,
            of.quarter_hour,
            of.time_period,
            of.order_id ,
            of.city_name, 
            of.pickup_location_hex_8,
            of.drop_location_hex_8,
            of.lm_flag,

            rp.data_orderid,
            rp.data_riderId,
            rp.Gross_orders,
            rp.riderBusy_pings,
            rp.riderreject_Pings,
            rp.accepted_pings,
            rp.already_accepted_pings,
            rp.rider_accept_failure_pings,
            rp.rider_ack_pings

        from order_filtering as of

        left join rider_pings as rp 

        on of.yyyymmdd = rp.yyyymmdd and of.order_id = rp.data_orderid

    """.format(std=i.strftime('%Y%m%d'))
    
    dt2=pd.read_sql_query(query, metabase_connection) 
    
    print(i)
    
    pings_dt1=pings_dt1.append(dt2)

2023-06-19 00:00:00


In [47]:
pings_dt1.head()

,yyyymmdd,quarter_hour,time_period,order_id,city_name,pickup_location_hex_8,drop_location_hex_8,lm_flag,data_orderid,data_riderId,Gross_orders,riderBusy_pings,riderreject_Pings,accepted_pings,already_accepted_pings,rider_accept_failure_pings,rider_ack_pings
0,20230619,0045,rest_morning,648f59c64559465ba91c2a50,Hyderabad,8860a259b9fffff,8860a259a1fffff,lm_greater_than_1km,648f59c64559465ba91c2a50,605e262c03f2a46d3020dbdf,1.0,0.0,0.0,0.0,2.0,0.0,2.0
1,20230619,0045,rest_morning,648f59c64559465ba91c2a50,Hyderabad,8860a259b9fffff,8860a259a1fffff,lm_greater_than_1km,648f59c64559465ba91c2a50,641896d1d4f7123bd1572a46,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,20230619,0045,rest_morning,648f59c64559465ba91c2a50,Hyderabad,8860a259b9fffff,8860a259a1fffff,lm_greater_than_1km,648f59c64559465ba91c2a50,5a0b0c2aee2c2d70c3838abe,1.0,1.0,0.0,0.0,1.0,0.0,2.0
3,20230619,0045,rest_morning,648f59c64559465ba91c2a50,Hyderabad,8860a259b9fffff,8860a259a1fffff,lm_greater_than_1km,648f59c64559465ba91c2a50,5f38e1c10fbf9678bc525d10,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,20230619,0045,rest_morning,648f59c64559465ba91c2a50,Hyderabad,8860a259b9fffff,8860a259a1fffff,lm_greater_than_1km,648f59c64559465ba91c2a50,5f3e4c15979c250436020e83,1.0,0.0,0.0,0.0,2.0,0.0,2.0


In [50]:
immu_data1=pd.DataFrame()
for i in pd.date_range('20230620','20230620'):

    query="""
    with order_filtering as 
    (
        select *,
        case when rounded_last_mile<=1 then 'lm_less_than_1km'
        else 'lm_greater_than_1km' end as lm_falg
        from
        (
        select distinct yyyymmdd,order_id ,city_name, pickup_location_hex_8,drop_location_hex_8,
            case when distance_final_distance<0.5 then round(distance_final_distance)+1 
            else round(distance_final_distance) end as rounded_last_mile,
                CASE 
                    WHEN substr(quarter_hour,1,2) IN ('08','09','10','11') THEN 'morning_peak'
                    WHEN substr(quarter_hour,1,2) IN ('12','13','14','15','16') THEN 'afternoon'
                    WHEN substr(quarter_hour,1,2) IN ('17','18','19','20','21') THEN 'evening_peak'
                    WHEN substr(quarter_hour,1,2) IN ('00','01','02','03','04','05','06','07') THEN 'rest_morning'        
                    ELSE 'rest_evening' END AS time_period, order_status,quarter_hour

        from 
            orders.order_logs_snapshot

            where yyyymmdd = '{std}'
            and service_obj_service_name = 'Auto'
            and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)
            and lower(trim(city_name)) in ('hyderabad') 
            )
    ),
    immutable as 
    (


                with immutabl_raw as (

                           select yyyymmdd, city_name,order_id,captain_id,event_type,order_status,updated_epoch,accept_to_pickup_distance

                            from orders.order_logs_immutable
                                where yyyymmdd = '{std}'
                                and lower(trim(city_name)) in ('bangalore')
                                and service_obj_service_name in ('Auto')
                                and order_id in (select distinct order_id from order_filtering)
                                and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)

                 ),


                immutable_v1 as

                        (
                            select yyyymmdd, city_name,order_id,captain_id,
                            count(distinct case when order_level_end_state_2='Net_Orders' then order_id end) as net_orders,
                            count(distinct case when order_level_end_state_2='OCARA_CC' then order_id end) as OCARA_CC,
                            count(distinct case when order_level_end_state_2='OCARA_RC' then order_id end) as OCARA_RC,
                            avg(accept_to_pickup_distance) as a2p

                            from
                            (
                            select yyyymmdd, city_name,order_id,captain_id,
                                case 
                                    when contains(order_status_agg, 'dropped') = true then 'Net_Orders'
                                    when contains(event_type_agg, 'accepted') = true and ( contains(order_status_agg, 'customerCancelled') =true and contains(event_type_agg, 'rider_cancelled') =false) then 'OCARA_CC'
                                    when contains(event_type_agg, 'accepted') = true and  (contains(event_type_agg, 'rider_cancelled') =true  and  contains(order_status_agg, 'customerCancelled') =false)then 'OCARA_RC'
                                    else 'others'
                                end as order_level_end_state_2,accept_to_pickup_distance
                            from
                            (
                                select * 
                                from

                                (
                                    select yyyymmdd, city_name,order_id,captain_id,accept_to_pickup_distance,
                                    array_agg(event_type) over (partition by order_id,captain_id) as event_type_agg,
                                    array_agg(order_status) over (partition by order_id,captain_id) as order_status_agg,
                                    row_number() over(partition by order_id,captain_id) as row_num

                                    from immutabl_raw

                                )
                                where row_num=1 
                            )
                            )
                            group by 1,2,3,4 

                    ),

           immutable_v2   as 

                        ( 
                            select yyyymmdd, city_name,order_id,
                            count(distinct order_id) as gross_orders,
                            count(distinct case when order_level_end_state='COBRA' then order_id end) as COBRA,
                            count(distinct case when order_level_end_state='Mapped_expiry' then order_id end) as Mapped_expiry,
                            count(distinct case when order_level_end_state='OCARA' then order_id end) as order_lvl_OCARA

                            from
                            (
                            select yyyymmdd, city_name,order_id,
                                case 
                                    when contains(event_type_agg, 'accepted') = false and  contains(order_status_agg, 'customerCancelled') =true then 'COBRA'
                                    when  contains(order_status_agg, 'expired') = true and contains(event_type_agg, 'accepted') = false then 'Mapped_expiry'
                                    when contains(event_type_agg, 'accepted') = true and  (contains(event_type_agg, 'rider_cancelled') =false and contains(event_type_agg, 'customer_cancelled') =true)  then 'OCARA'
                                    when contains(event_type_agg, 'accepted') = true and  (contains(event_type_agg, 'rider_cancelled') =true and contains(event_type_agg, 'customer_cancelled') =false)  then 'OCARA'                                
                                    else 'others'
                                end as order_level_end_state
                            from
                            (
                                select * 
                                from

                                (
                                    select yyyymmdd, city_name,order_id,
                                    array_agg(event_type) over (partition by order_id) as event_type_agg,
                                    array_agg(order_status) over (partition by order_id) as order_status_agg,
                                    row_number() over(partition by order_id) as row_num

                                    from immutabl_raw

                                )
                                where row_num=1 
                            )
                            )
                            group by 1,2,3 

                            ),

                tta_dt as     (


                            select  yyyymmdd,order_id,
                            ((acc_epoch - req_epoch)/1000.0) as tta,
                            ((max_acc_epoch - req_epoch)/1000.0) as tta_max,
                            ((acc_epoch_cancel - req_epoch)/1000.0) as ttc,
                            ((max_acc_epoch_cancel - req_epoch)/1000.0) as ttc_max 

                            from
                            (
                                 select  yyyymmdd,order_id,
                                min(case when event_type='order_requested' then updated_epoch end) as req_epoch, 
                                min(case when event_type='accepted' then updated_epoch end) as acc_epoch,
                                max(case when event_type='accepted' then updated_epoch end) as max_acc_epoch,
                                min(case when event_type='rider_cancelled' then updated_epoch end) as acc_epoch_cancel,
                                max(case when event_type='rider_cancelled' then updated_epoch end) as max_acc_epoch_cancel

                                from immutabl_raw

                                group by 1,2 

                                )
                            )


        select iv1.yyyymmdd,iv1.order_id ,iv1.city_name,
        captain_id, gross_orders,COBRA,Mapped_expiry ,net_orders,order_lvl_OCARA,OCARA_CC, OCARA_RC ,
        tta,tta_max,ttc,ttc_max,a2p 

        from immutable_v1 as iv1 

        left join immutable_v2 as iv2  

        on iv1.yyyymmdd = iv2.yyyymmdd and iv1.order_id = iv2.order_id


        left join tta_dt

        on iv1.yyyymmdd = tta_dt.yyyymmdd and iv1.order_id = tta_dt.order_id

        order by order_id,captain_id 
        )


        select * from
        immutable


        """.format(std=i.strftime('%Y%m%d'))
    
    immu_dt1=pd.read_sql_query(query, metabase_connection) 
    
    print(i)
    
    immu_data1=immu_data1.append(immu_dt1)


2023-06-20 00:00:00


In [52]:
immu_dt1.head()

,yyyymmdd,order_id,city_name,captain_id,gross_orders,COBRA,Mapped_expiry,net_orders,order_lvl_OCARA,OCARA_CC,OCARA_RC,tta,tta_max,ttc,ttc_max,a2p


In [45]:
res_data1=pd.DataFrame()
for i in pd.date_range('20230620','20230620'):


    query="""with order_filtering as 
    (
        select *,
        case when rounded_last_mile<=1 then 'lm_less_than_1km'
        else 'lm_greater_than_1km' end as lm_falg
        from
        (
        select distinct yyyymmdd,order_id ,city_name, pickup_location_hex_8,drop_location_hex_8,
            case when distance_final_distance<0.5 then round(distance_final_distance)+1 
            else round(distance_final_distance) end as rounded_last_mile,
                CASE 
                    WHEN substr(quarter_hour,1,2) IN ('08','09','10','11') THEN 'morning_peak'
                    WHEN substr(quarter_hour,1,2) IN ('12','13','14','15','16') THEN 'afternoon'
                    WHEN substr(quarter_hour,1,2) IN ('17','18','19','20','21') THEN 'evening_peak'
                    WHEN substr(quarter_hour,1,2) IN ('00','01','02','03','04','05','06','07') THEN 'rest_morning'        
                    ELSE 'rest_evening' END AS time_period, order_status,quarter_hour

        from 
            orders.order_logs_snapshot

            where yyyymmdd = '{std}'
            and service_obj_service_name = 'Auto'
            and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)
            and lower(trim(city_name)) in ('hyderabad') 
            )
    )

    select city_name,yyyymmdd,order_id,rider_id,fm,
                 case when fm <= 500 then 'fm_<_500m'
                      when fm <= 1000 then '500_<fm<_1km'
                      else 'more_than_1km' end as fm_buckets

            from
            (
             select 
                    city_name,yyyymmdd,order_id, 
                    rider_id, cast(fm as real) as fm ,
                    row_number() over (partition by order_id, rider_id order by epoch_timestamp) as Rank_v1

            from
                (
                    select 
                    city_name,yyyymmdd,order_id,last_mile_distance as lm,
                    updated_at as epoch_timestamp,
                    transform(cast(captains_with_score as array<json>), x->json_extract_scalar(x,'$.captain_id')) as rider_list_1,
                    transform(cast(captains_with_score as array<json>), x->json_extract_scalar(x,'$.first_mile_distance')) as fm_1
                    from g2n.idispatch_api_responses 
                    where yyyymmdd = '{std}'
                    and lower(trim(service_category)) = 'auto'
                    and order_id in (select distinct order_id from order_filtering)

            )
            cross join unnest(rider_list_1, fm_1) as t(rider_id, fm)

        )
        where Rank_v1=1

        """.format(std=i.strftime('%Y%m%d'))
    

    response_dt=pd.read_sql_query(query, metabase_connection) 
    
    print(i)
    
    res_data1=res_data1.append(response_dt)


    
    
    

2023-06-20 00:00:00


In [49]:
response_dt.head()

,city_name,yyyymmdd,order_id,rider_id,fm,fm_buckets
0,Hyderabad,20230620,64916382698cf47c3dd7c1a5,61b038dbf3de6f3201461e4f,1576.3,more_than_1km
1,Hyderabad,20230620,64916384e205f47068796bf4,61d9aa17d80f2bb8eaa22eae,1425.6,more_than_1km
2,Hyderabad,20230620,649161948e0cce01250c41f8,628b0b78cf1a86eb9711cd46,242.4,fm_<_500m
3,Hyderabad,20230620,649161948e0cce01250c41f8,6433efb848a90f616793653a,919.4,500_<fm<_1km
4,Hyderabad,20230620,649161948e0cce01250c41f8,627a935f87b0a6bd2cc52532,1840.3,more_than_1km


In [ ]:
response_dt.info()

In [ ]:
len(response_dt)

In [ ]:
pings_dt1.shape

In [ ]:
#pings_dt1[pings_dt1.order_id=='6483767e18193c64c9ffa669']

In [ ]:
pings_dt1.head()

In [ ]:
immu_dt1.head()

In [ ]:
pings_dt1=pings_dt1.rename(columns={'data_riderId':'captain_id'})

In [ ]:
pings_dt1=pings_dt1.drop_duplicates(subset=['data_orderid','captain_id']).reset_index(drop=True)

In [ ]:
first_cut=pings_dt1.merge(immu_dt1,on=['city_name','yyyymmdd','order_id','captain_id'],how='left')

In [ ]:
first_cut.head()

In [ ]:
response_dt=response_dt.rename(columns={'rider_id':'captain_id'})

In [ ]:
second_cut=first_cut.merge(response_dt,how='left',on=['city_name','yyyymmdd','order_id','captain_id'])

In [ ]:
second_cut.head()

In [ ]:
second_cut.columns

In [138]:
#second_cut.to_parquet('as_of_10th_data.parquet',index=False)

In [ ]:
second_cut.to_csv('as_of_19th_data_hyd.csv',index=False)

In [36]:
#second_cut=pd.read_csv('as_of_19th_data.csv')

In [37]:
second_cut.head()

,yyyymmdd,quarter_hour,time_period,order_id,city_name,pickup_location_hex_8,drop_location_hex_8,lm_flag,data_orderid,captain_id,...,order_lvl_OCARA,OCARA_CC,OCARA_RC,tta,tta_max,ttc,ttc_max,a2p,fm,fm_buckets
0,20230620,2230,rest_evening,6491dd382b1dca469bf346e8,Bangalore,88618925d9fffff,88618925ddfffff,lm_greater_than_1km,6491dd382b1dca469bf346e8,6308bce550f57e5b7cfe0631,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,896.7,500_<fm<_1km
1,20230620,2230,rest_evening,6491dd382b1dca469bf346e8,Bangalore,88618925d9fffff,88618925ddfffff,lm_greater_than_1km,6491dd382b1dca469bf346e8,5cc96ebf6e29796012c2e3bd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,707.4,500_<fm<_1km
2,20230620,2230,rest_evening,6491dd382b1dca469bf346e8,Bangalore,88618925d9fffff,88618925ddfffff,lm_greater_than_1km,6491dd382b1dca469bf346e8,62aaae4c7b34ac0bdf26f5a7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,798.0,500_<fm<_1km
3,20230620,2230,rest_evening,6491dd382b1dca469bf346e8,Bangalore,88618925d9fffff,88618925ddfffff,lm_greater_than_1km,6491dd382b1dca469bf346e8,61b32a035499d38642a29919,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.5,fm_<_500m
4,20230620,2230,rest_evening,6491dd382b1dca469bf346e8,Bangalore,88618925d9fffff,88618925ddfffff,lm_greater_than_1km,6491dd382b1dca469bf346e8,640b177e27372397b48ed30b,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,868.6,500_<fm<_1km


In [38]:
#gross orders
second_cut.groupby(['yyyymmdd','city_name','lm_flag','fm_buckets'],as_index=False)[['order_id']].nunique().to_clipboard()



In [39]:
second_cut.groupby(['yyyymmdd','city_name','lm_flag','fm_buckets'])[['riderBusy_pings','riderreject_Pings','accepted_pings',
        'already_accepted_pings','rider_accept_failure_pings','rider_ack_pings','net_orders','OCARA_CC','OCARA_RC']].sum().to_clipboard()



### time period level

In [ ]:
#apr,dpr,dapr,ocara,

In [40]:
#gross orders
second_cut.groupby(['yyyymmdd','city_name','time_period','lm_flag','fm_buckets'],as_index=False)[['order_id']].nunique().to_clipboard()



In [41]:
second_cut.groupby(['yyyymmdd','city_name','time_period','lm_flag','fm_buckets'],as_index=False)[['riderBusy_pings','riderreject_Pings','accepted_pings',
        'already_accepted_pings','rider_accept_failure_pings','rider_ack_pings','net_orders','OCARA_CC','OCARA_RC']].sum().to_clipboard()



In [34]:


# """
# with order_filtering as 
# (
#     select *,
#     case when rounded_last_mile<=1 then 'lm_less_than_1km'
#     else 'lm_greater_than_1km' end as lm_falg
#     from
#     (
#     select distinct yyyymmdd,order_id ,city_name, pickup_location_hex_8,drop_location_hex_8,
#         distance_final_distance,
#         case when distance_final_distance<0.5 then round(distance_final_distance)+1 
#         else round(distance_final_distance) end as rounded_last_mile,
#             CASE 
#                 WHEN substr(quarter_hour,1,2) IN ('08','09','10','11') THEN 'morning_peak'
#                 WHEN substr(quarter_hour,1,2) IN ('12','13','14','15','16') THEN 'afternoon'
#                 WHEN substr(quarter_hour,1,2) IN ('17','18','19','20','21') THEN 'evening_peak'
#                 WHEN substr(quarter_hour,1,2) IN ('00','01','02','03','04','05','06','07') THEN 'rest_morning'        
#                 ELSE 'rest_evening' END AS time_period, order_status,quarter_hour

#     from 
#         orders.order_logs_snapshot
#         --where yyyymmdd = cast({dt} as varchar)
#         where yyyymmdd = '20230610'
#         and service_obj_service_name = 'Auto'
#         and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)
#         and lower(trim(city_name)) in ('bangalore') 
#         )
# ),
# immutable as 
# (


#             with immutabl_raw as (

#                        select yyyymmdd, city_name,order_id,captain_id,event_type,order_status,updated_epoch,accept_to_pickup_distance

#                         from orders.order_logs_immutable
#                             where yyyymmdd = '20230610'
#                             and lower(trim(city_name)) in ('bangalore')
#                             and service_obj_service_name in ('Auto')
#                             and order_id in (select distinct order_id from order_filtering)
#                             and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)

#              ),
   

#             immutable_v1 as

#                     (
#                         select yyyymmdd, city_name,order_id,captain_id,
#                         count(distinct case when order_level_end_state_2='Net_Orders' then order_id end) as net_orders,
#                         count(distinct case when order_level_end_state_2='OCARA_CC' then order_id end) as OCARA_CC,
#                         count(distinct case when order_level_end_state_2='OCARA_RC' then order_id end) as OCARA_RC,
#                         avg(accept_to_pickup_distance) as a2p
                        
#                         from
#                         (
#                         select yyyymmdd, city_name,order_id,captain_id,
#                             case 
#                                 when contains(order_status_agg, 'dropped') = true then 'Net_Orders'
#                                 when contains(event_type_agg, 'accepted') = true and ( contains(order_status_agg, 'customerCancelled') =true and contains(event_type_agg, 'rider_cancelled') =false) then 'OCARA_CC'
#                                 when contains(event_type_agg, 'accepted') = true and  (contains(event_type_agg, 'rider_cancelled') =true  and  contains(order_status_agg, 'customerCancelled') =false)then 'OCARA_RC'
#                                 else 'others'
#                             end as order_level_end_state_2,accept_to_pickup_distance
#                         from
#                         (
#                             select * 
#                             from

#                             (
#                                 select yyyymmdd, city_name,order_id,captain_id,accept_to_pickup_distance,
#                                 array_agg(event_type) over (partition by order_id,captain_id) as event_type_agg,
#                                 array_agg(order_status) over (partition by order_id,captain_id) as order_status_agg,
#                                 row_number() over(partition by order_id,captain_id) as row_num

#                                 from immutabl_raw

#                             )
#                             where row_num=1 
#                         )
#                         )
#                         group by 1,2,3,4 

#                 ),
                    
#        immutable_v2   as 
                    
#                     ( 
#                         select yyyymmdd, city_name,order_id,
#                         count(distinct order_id) as gross_orders,
#                         count(distinct case when order_level_end_state='COBRA' then order_id end) as COBRA,
#                         count(distinct case when order_level_end_state='Mapped_expiry' then order_id end) as Mapped_expiry,
#                         count(distinct case when order_level_end_state='OCARA' then order_id end) as order_lvl_OCARA
                        
#                         from
#                         (
#                         select yyyymmdd, city_name,order_id,
#                             case 
#                                 when contains(event_type_agg, 'accepted') = false and  contains(order_status_agg, 'customerCancelled') =true then 'COBRA'
#                                 when  contains(order_status_agg, 'expired') = true and contains(event_type_agg, 'accepted') = false then 'Mapped_expiry'
#                                 when contains(event_type_agg, 'accepted') = true and  (contains(event_type_agg, 'rider_cancelled') =false and contains(event_type_agg, 'customer_cancelled') =true)  then 'OCARA'
#                                 when contains(event_type_agg, 'accepted') = true and  (contains(event_type_agg, 'rider_cancelled') =true and contains(event_type_agg, 'customer_cancelled') =false)  then 'OCARA'                                
#                                 else 'others'
#                             end as order_level_end_state
#                         from
#                         (
#                             select * 
#                             from

#                             (
#                                 select yyyymmdd, city_name,order_id,
#                                 array_agg(event_type) over (partition by order_id) as event_type_agg,
#                                 array_agg(order_status) over (partition by order_id) as order_status_agg,
#                                 row_number() over(partition by order_id) as row_num

#                                 from immutabl_raw

#                             )
#                             where row_num=1 
#                         )
#                         )
#                         group by 1,2,3 
    
#                         ),
                        
#             tta_dt as     (
            
            
#                         select  yyyymmdd,order_id,
#                         ((acc_epoch - req_epoch)/1000.0) as tta,
#                         ((max_acc_epoch - req_epoch)/1000.0) as tta_max,
#                         ((acc_epoch_cancel - req_epoch)/1000.0) as ttc,
#                         ((max_acc_epoch_cancel - req_epoch)/1000.0) as ttc_max 
                        
#                         from
#                         (
#                              select  yyyymmdd,order_id,
#                             min(case when event_type='order_requested' then updated_epoch end) as req_epoch, 
#                             min(case when event_type='accepted' then updated_epoch end) as acc_epoch,
#                             max(case when event_type='accepted' then updated_epoch end) as max_acc_epoch,
#                             min(case when event_type='rider_cancelled' then updated_epoch end) as acc_epoch_cancel,
#                             max(case when event_type='rider_cancelled' then updated_epoch end) as max_acc_epoch_cancel
                            
#                             from immutabl_raw
                            
#                             group by 1,2 
                        
#                             )
#                         )
                        
                        
#     select iv1.yyyymmdd,iv1.order_id ,iv1.city_name,
#     captain_id, gross_orders,COBRA,Mapped_expiry ,net_orders,order_lvl_OCARA,OCARA_CC, OCARA_RC ,
#     tta,tta_max,ttc,ttc_max,a2p 
    
#     from immutable_v1 as iv1 

#     left join immutable_v2 as iv2  

#     on iv1.yyyymmdd = iv2.yyyymmdd and iv1.order_id = iv2.order_id
    
    
#     left join tta_dt
    
#     on iv1.yyyymmdd = tta_dt.yyyymmdd and iv1.order_id = tta_dt.order_id
        
#     order by order_id,captain_id 
#     )
     
# select  of.yyyymmdd,
#         of.quarter_hour,
#         of.time_period,
#         of.order_id ,
#         of.city_name, 
#         of.pickup_location_hex_8,
#         of.drop_location_hex_8,
#         of.lm_falg,
        
#         rp.data_orderid,
#         rp.data_riderId,
#         rp.Gross_orders,
#         rp.riderBusy_pings,
#         rp.riderreject_Pings,
#         rp.accepted_pings,
#         rp.already_accepted_pings,
#         rp.rider_accept_failure_pings,
#         rp.rider_ack_pings,
        
#         imm.net_orders,
#         imm.COBRA,
#         imm.Mapped_expiry,
#         imm.order_lvl_OCARA,
#         imm.OCARA_CC, 
#         imm.OCARA_RC ,
#         imm.tta,
#         imm.tta_max,
#         imm.ttc,
#         imm.ttc_max       
        
#         --ard.fm,
#         --ard.fm_buckets
        

# from order_filtering as of

# left join rider_pings as rp 
    
# on of.yyyymmdd = rp.yyyymmdd and of.order_id = rp.data_orderid
    
# left join immutable as imm 

# on  rp.data_orderid =imm.order_id and rp.data_riderId=imm.captain_id and rp.yyyymmdd = imm.yyyymmdd

# --left join api_res_data as ard 

# --on  rp.yyyymmdd= ard.yyyymmdd and rp.data_orderid = ard.order_id and rp.data_riderId= ard.rider_id

# """
    

# dt1=pd.read_sql_query(query, metabase_connection) 



In [ ]:
"""query=


with t1 as (
        select distinct yyyymmdd,order_id,distance_final_distance,order_status,
        case when distance_final_distance<0.5 then round(distance_final_distance)+1 
            else round(distance_final_distance) end as rounded_last_mile
    from 
        orders.order_logs_snapshot
        where yyyymmdd >= '20230601' and yyyymmdd <= '20230610'
        and service_obj_service_name = 'Auto'
        and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)
        and city_name in ('Bangalore')
        
    ),
    
t2 as (   

    select yyyymmdd,count(distinct order_id) less_1km_orders
    from
    t1    
    where distance_final_distance<=1
    group by 1
    
    ),
    
t3 as (   

    select yyyymmdd,count(distinct order_id) as  more_than_1km_orders
    from
    t1    
    where distance_final_distance>1
    group by 1
    ),

t4 as (   

    select yyyymmdd,count(distinct order_id) less_1km_orders_dropped
    from
    t1    
    where distance_final_distance<=1 and order_status='dropped'
    group by 1
    
    )
    
    select t2.yyyymmdd,less_1km_orders,more_than_1km_orders,less_1km_orders_dropped
    
    from t2
    left join t3
    on t2.yyyymmdd=t3.yyyymmdd
    
    left join t4
    on t2.yyyymmdd=t4.yyyymmdd    
    
    
"""
dt1=pd.read_sql_query(query, metabase_connection)